In [1]:
!nvidia-smi

Sat Apr 17 19:02:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 435.21       Driver Version: 435.21       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:04:00.0 Off |                  N/A |
| 31%   34C    P0    67W / 250W |      0MiB / 11019MiB |     17%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:05:00.0 Off |                  N/A |
| 30%   19C    P8     4W / 250W |      0MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
train_fp = r'training/labeled'
unlabeled_fp = r'training/unlabeled'
vaild_fp = r'validation'
test_fp = r'training/unlabeled'

In [8]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import torchvision
from torch.utils.data import ConcatDataset, DataLoader, TensorDataset
from torchvision.datasets import DatasetFolder
from tqdm.auto import tqdm

train_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.,0.,0.],[1.,1.,1.])

])


test_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.,0.,0.],[1.,1.,1.])

])




train_set = DatasetFolder(train_fp, loader = lambda x: Image.open(x),
                          extensions = "jpg", transform = train_tfm)
valid_set = DatasetFolder(vaild_fp, loader = lambda x: Image.open(x),
                          extensions = "jpg", transform = test_tfm)
unlabeled_set = DatasetFolder(unlabeled_fp,
                              loader = lambda x: Image.open(x), extensions = "jpg", transform = train_tfm)
test_set = DatasetFolder(test_fp, loader = lambda x: Image.open(x),
                         extensions = "jpg", transform = test_tfm)

train_loader = DataLoader(train_set, batch_size = 68, shuffle = True, )
valid_loader = DataLoader(valid_set, batch_size = 250, shuffle = True, )
test_loader = DataLoader(test_set, batch_size = 250, shuffle = False)


class contact_set(torch.utils.data.Dataset):
    def __init__(self, old_set, new_set):
        self.old_set = old_set
        self.new_set = new_set

    def __len__(self):
        return len(self.old_set) + len(self.new_set)

    def __getitem__(self, idx):
        if idx < len(self.old_set):
            return self.old_set[idx]
        else:
            return self.new_set[idx - len(self.old_set)]
model = nn.Sequential(torchvision.models.resnet18(pretrained = True),
                     nn.Linear(1000, 200),
                     nn.SELU(),
                     nn.Linear(200, 11))

def get_pseudo_label(old_set, unlabeled_set, model, batch_size = 350, threshold = 0.7):
    data_loader = DataLoader(unlabeled_set, batch_size)
    model.eval()
    softmax = nn.Softmax(-1)
    for img_batch, _ in tqdm(data_loader):
        with torch.no_grad():
            logits = model(img_batch.cuda()).cpu()
        probs = softmax(logits)
        probs_max_bs_1, pos_max_bs_1 = probs.max(-1)
        bool_index = probs_max_bs_1 >= threshold
        new_data = img_batch[bool_index]
        new_label = pos_max_bs_1[bool_index]
        new_set = TensorDataset(new_data, new_label)
        old_set = contact_set(old_set, new_set)
    return old_set


device = "cuda"

model = model.cuda()
model.device = device

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4, weight_decay = 1e-5)
n_epochs = 200

for epoch in range(n_epochs):
    if epoch > 3:

        new_set = get_pseudo_label( train_set,unlabeled_set, model)
        print('New set len: ', len(new_set) - len(train_set))
        train_loader = DataLoader(new_set, batch_size = 300)

    model.train()

    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):
        imgs, labels = batch
        logits = model(imgs.to(device))
        loss = criterion(logits, labels.to(device))
        optimizer.zero_grad()
        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm = 10)
        optimizer.step()
        acc = (logits.argmax(dim = -1) == labels.to(device)).float().mean()
        train_loss.append(loss.item())
        train_accs.append(acc)

    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    model.eval()

    valid_loss = []
    valid_accs = []

    for batch in tqdm(valid_loader):
        imgs, labels = batch
        with torch.no_grad():
            logits = model(imgs.to(device))
        loss = criterion(logits, labels.to(device))
        acc = (logits.argmax(dim = -1) == labels.to(device)).float().mean()
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

model.eval()

predictions = []

for batch in tqdm(test_loader):
    imgs, labels = batch
    with torch.no_grad():
        logits = model(imgs.to(device))
    predictions.extend(logits.argmax(dim = -1).cpu().numpy().tolist())

with open("predict.csv", "w") as f:
    f.write("Id,Category\n")
    for i, pred in enumerate(predictions):
        f.write(f"{i},{pred}\n")


[ Train | 001/200 ] loss = 0.85429, acc = 0.71503



[ Valid | 001/200 ] loss = 0.93504, acc = 0.74542



[ Train | 002/200 ] loss = 0.19388, acc = 0.93689



[ Valid | 002/200 ] loss = 0.94597, acc = 0.76692



[ Train | 003/200 ] loss = 0.06476, acc = 0.97673


...


[ Valid | 020/200 ] loss = 3.99026, acc = 0.41133



New set len:  6577



[ Train | 021/200 ] loss = 0.06946, acc = 0.98091



[ Valid | 021/200 ] loss = 3.87935, acc = 0.38500


KeyboardInterrupt: 